In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model
os.chdir("C:\graduationProject2023")
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
class point():
    def __init__(self, x, y, z):
        self.x = x 
        self.y = y
        self.z = z
        
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
def calculate_distance(point1, point2):
    x = abs(point1.x - point2.x)
    y = abs(point1.y - point2.y)
    z = abs(point1.z - point2.z)
    p = point(x,y,z)
    return p

def extract_midpoint(results):
    try:
        left_shoulder = results.pose_landmarks.landmark[11]
        right_shoulder = results.pose_landmarks.landmark[12]
        x = (left_shoulder.x + right_shoulder.x)/2
        y = (left_shoulder.y + right_shoulder.y)/2
        z = (left_shoulder.z + right_shoulder.z)/2
        visiblity = max(left_shoulder.visibility, right_shoulder.visibility)
        mid_point = point(x,y,z)
        return mid_point
    except:
        pass
    
def extract_keypoints(results, midpoint): 
#     for(res in results.pose_landmarks.landmark):
#         point = point(res.x,res.y,res.z)
#         distances = calculate_distance(point, midpoint)
    
    # POSE LANDMARKS
    if results.pose_landmarks:
        pose = []
        for res in results.pose_landmarks.landmark:
            p = point(res.x, res.y, res.z)
            distances = calculate_distance(p, midpoint)
            pose.extend([distances.x, distances.y, distances.z])
        pose = np.array(pose).flatten()
    else:
        pose = np.zeros(33*3)
    
    # LEFT HAND LANDMARKS
    if results.left_hand_landmarks:
        lh = []
        for res in results.left_hand_landmarks.landmark:
            p = point(res.x, res.y, res.z)
            distances = calculate_distance(p, midpoint)
            lh.extend([distances.x, distances.y, distances.z])
        lh = np.array(lh).flatten()
    else:
        lh = np.zeros(21*3)

    # RIGHT HAND LANDMARKS
    if results.right_hand_landmarks:
        rh = []
        for res in results.right_hand_landmarks.landmark:
            p = point(res.x, res.y, res.z)
            distances = calculate_distance(p, midpoint)
            rh.extend([distances.x, distances.y, distances.z])
        rh = np.array(rh).flatten()
    else:
        rh = np.zeros(21*3)

    return np.concatenate([pose, lh, rh])

actions = np.array(['dance','hello','how','iloveyou','meet','morning','please','thanks','thirsty','use','you'])

label_map = {label:num for num, label in enumerate(actions)}
model = load_model('dynamic_50epochs.h5')
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.95, min_tracking_confidence=0.95) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        midpoint = extract_midpoint(results)
        keypoints = extract_keypoints(results,midpoint)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti